<a href="https://www.kaggle.com/code/matheus7cassiano/noir-fraud-risk-intelligence?scriptVersionId=290197758" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


# 🌑 NOIR - NETWORK-ORIENTED INTELLIGENCE FOR RISK (EM DESENVOLVIMENTO)
> Sistema Anti-Fraude de Última Geração
> 
> Missão: Romper as raízes de esquemas criminosos financeiros através de análise de redes e IA
> Diferencial: Não detectamos transações - DESMANTELAMOS QUADRILHAS INTEIRAS



> IMPORTS & CONFIGURAÇÃO INICIAL

In [1]:
# Setup de Infraestrutura e Bibliotecas de Análise
import pandas as pd
import numpy as np  
import os

# Definição do path para o Data Lake do Kaggle
csv = '/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv'
print("🚀 SYSTEM BOOT: Iniciando ingestão de dados no Noir...")

🚀 SYSTEM BOOT: Iniciando ingestão de dados no Noir...


> IMPORTAÇÃO E ENRIQUECIMENTO DO PAYSIM1

In [2]:
print("="*80)
print("📥 IMPORTANDO DATASET PAYSIM1 (MODO PRO)...")
print("="*80)

df_raw = pd.read_csv(csv)

# 2. FILTRAR APENAS TRANSFER E CASH_OUT (Onde realmente acontecem os golpes)
df_raw = df_raw[df_raw['type'].isin(['TRANSFER', 'CASH_OUT'])]

print("🎲 Realizando Amostragem Baseada em Contas...")
all_users = df_raw['nameOrig'].unique()
# Peguei 20.000 usuários aleatórios e trouxe  TODO o histórico deles
sampled_users = np.random.choice(all_users, 20000, replace=False)
df_sample = df_raw[df_raw['nameOrig'].isin(sampled_users)].copy()

print(f"✅ Dataset Filtrado: {df_sample.shape} transações de {len(sampled_users)} usuários únicos.")


print("🔧 CONSTRUINDO PERFIL CRIMINAL DAS CONTAS...")

# Agregação Poderosa
df_accounts = df_sample.groupby('nameOrig').agg({
    'step': ['min', 'max', 'count'],       # Tempo e Frequência
    'amount': ['sum', 'mean', 'std', 'max'], # Volume e Variabilidade
    'nameDest': 'nunique',                 # Entropia de Destino
    'isFraud': 'max',                      # Se já foi pego
    'type': lambda x: (x == 'CASH_OUT').sum() # Preferência por Saque
}).reset_index()

df_accounts.columns = [
    'account_id', 'first_step', 'last_step', 'n_transactions',
    'total_amount', 'avg_amount', 'std_amount', 'max_amount',
    'n_unique_beneficiaries', 'is_fraud', 'n_cashouts'
]

df_accounts['std_amount'] = df_accounts['std_amount'].fillna(0)


df_accounts['activity_window_hours'] = df_accounts['last_step'] - df_accounts['first_step']

# Burn Rate: Dinheiro por Hora
# (Adicionei +0.1 pra evitar divisão por zero em transações simultâneas)
df_accounts['burn_rate'] = df_accounts['total_amount'] / (df_accounts['activity_window_hours'] + 0.1)

# Cash Residence (Lógica ajustada)
# Se o cara movimenta muito em janela zero (tudo na mesma hora), o residence time é quase zero.
df_accounts['cash_residence_hours'] = np.where(
    df_accounts['n_transactions'] > 1,
    df_accounts['activity_window_hours'] / df_accounts['n_transactions'],
    0.1 # Se fez só 1 transação, assumimos que foi "vapt-vupt" se for fraude
)

df_accounts['beneficiary_entropy'] = df_accounts['n_unique_beneficiaries'] / df_accounts['n_transactions']



n_accs = len(df_accounts)

dirty_devices = [f"DEV_DIRTY_{i}" for i in range(50)] 

clean_devices = [f"DEV_CLEAN_{i}" for i in range(n_accs)]

def assign_device(row):
    
    if row['is_fraud'] == 1:
        return np.random.choice(dirty_devices) if np.random.random() < 0.85 else f"DEV_UNIQUE_{np.random.randint(99999)}"
    
    else:
        return np.random.choice(clean_devices) if np.random.random() < 0.99 else np.random.choice(dirty_devices)

df_accounts['device_id'] = df_accounts.apply(assign_device, axis=1)


device_counts = df_accounts.groupby('device_id')['account_id'].count().to_dict()
df_accounts['device_sharing_score'] = df_accounts['device_id'].map(device_counts)


df_accounts['dormancy_score'] = np.where(
    (df_accounts['activity_window_hours'] <= 1) & (df_accounts['total_amount'] > 100000), 
    1.0, 0.0
)

df_accounts['crypto_exposure'] = df_accounts['n_cashouts'] / df_accounts['n_transactions']


df_accounts['fraud_network_distance'] = 0 
df_accounts['fraud_type'] = np.where(df_accounts['is_fraud'] == 1, 'account_takeover', 'legit')

df = df_accounts.copy()

print(f"✅ PERFILAMENTO CONCLUÍDO!")
print(f"📊 Total de Contas Únicas: {len(df)}")
print(f"🚨 Contas Comprometidas: {df['is_fraud'].sum()}")
print("\n🔍 Amostra do NOIR:")
display(df.T)

📥 IMPORTANDO DATASET PAYSIM1 (MODO PRO)...
🎲 Realizando Amostragem Baseada em Contas...
✅ Dataset Filtrado: (20016, 11) transações de 20000 usuários únicos.
🔧 CONSTRUINDO PERFIL CRIMINAL DAS CONTAS...
✅ PERFILAMENTO CONCLUÍDO!
📊 Total de Contas Únicas: 20000
🚨 Contas Comprometidas: 65

🔍 Amostra do NOIR:


,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
account_id,C1000094846,C1000112658,C1000197677,C1000203399,C100033456,C1000352821,C1000455215,C1000760563,C1000887041,C1000900325,...,C998913833,C999163325,C999239195,C999418285,C99943396,C999685961,C999705796,C999875531,C99988576,C999992757
first_step,257,39,259,38,186,592,165,395,11,376,...,163,14,154,347,256,404,332,20,37,428
last_step,257,39,259,38,186,592,165,395,11,376,...,163,14,154,347,256,404,332,20,37,428
n_transactions,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
total_amount,835095.2,1850234.86,168236.9,99747.16,33639.16,14135.91,241931.22,249774.82,394691.85,137721.1,...,261858.46,331933.97,193944.44,1750604.66,79217.51,107179.87,1586105.19,295257.87,15082.52,1085839.38
avg_amount,835095.2,1850234.86,168236.9,99747.16,33639.16,14135.91,241931.22,249774.82,394691.85,137721.1,...,261858.46,331933.97,193944.44,1750604.66,79217.51,107179.87,1586105.19,295257.87,15082.52,1085839.38
std_amount,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max_amount,835095.2,1850234.86,168236.9,99747.16,33639.16,14135.91,241931.22,249774.82,394691.85,137721.1,...,261858.46,331933.97,193944.44,1750604.66,79217.51,107179.87,1586105.19,295257.87,15082.52,1085839.38
n_unique_beneficiaries,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
is_fraud,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
